# Cargadores de documentos en LangChain


LangChain incluye cargadores integrados que permiten leer archivos y convertirlos rápidamente en objetos del tipo `Document`, listos para su procesamiento dentro del flujo de trabajo de la biblioteca.

### Objeto `Document` en LangChain

El objeto `Document` es una estructura fundamental en LangChain que representa una unidad de texto junto con sus metadatos asociados. 

- **Contenido principal (`page_content`)**: Texto o información que se desea procesar o analizar.
- **Metadatos (`metadata`)**: Información adicional contextual que puede incluir el origen del documento, número de página, etiquetas, etc.

Este diseño facilita la manipulación y el seguimiento de fragmentos de texto en flujos de trabajo con modelos de lenguaje, permitiendo almacenar tanto el contenido como datos relevantes que mejoran la gestión y el análisis.

---


### Características de los cargadores
- Permiten cargar documentos desde múltiples formatos como PDF, HTML, TXT, entre otros.
- Transforman el contenido en una estructura uniforme que LangChain puede utilizar directamente.

### Dependencias externas
Muchos cargadores requieren bibliotecas adicionales. Por ejemplo:

- Para archivos PDF, se necesita la biblioteca `pypdf`.
- Para archivos HTML, se requiere `beautifulsoup4`.

Es importante asegurarse de instalar las bibliotecas necesarias antes de usar cada cargador. Si una dependencia falta, LangChain mostrará un mensaje indicando cuál debes instalar.

### Documentación oficial
Puedes consultar la lista completa de cargadores disponibles en la siguiente dirección:

https://python.langchain.com/v0.2/docs/integrations/document_loaders/


Nota: Es posible que se deba instalar alguna libreria
Instalar la librería de langChain Community:
- En cmd: pip install langchain-community

## Carga de datos CSV


CSVLoader es un cargador de documentos en LangChain que permite leer archivos .csv y convertir cada fila (o el archivo completo) en uno o varios objetos Document.

#### Características clave:
- Lee archivos CSV delimitados por comas u otros caracteres.
- Por defecto, cada fila del CSV se convierte en un documento independiente.
- Puedes configurar qué columna usar como contenido principal (page_content_column) y cuáles como metadatos.
- Útil para datos tabulares: reportes, logs, registros estructurados.

El `CSVLoader` de LangChain facilita la integración directa de archivos CSV en flujos de trabajo con modelos de lenguaje, ya que convierte automáticamente cada fila en un documento estructurado con contenido principal y metadatos, lo que simplifica la manipulación, búsqueda y análisis de datos tabulares. Esto permite aprovechar de manera eficiente la información contenida en CSVs para tareas de procesamiento de lenguaje natural, evitando la necesidad de preprocesamientos complejos y garantizando una compatibilidad óptima con las herramientas y componentes del ecosistema LangChain.

In [ ]:
# pip install langchain-community en una terminal. 

from langchain.document_loaders import CSVLoader

In [ ]:
# cargamos un fichero CSV

loader = CSVLoader(
    file_path="Datos/Datos_Empleados.csv",
    encoding='utf-8',  # probar con 'latin-1' si se ven caracteres raros
    )


In [ ]:
# Se crea el objeto "data" con los datos desde el cargador "loader"

data = loader.load()

In [ ]:
 print(data) # vemos que se ha cargado un documento por cada fila donde el campo page_content contiene los datos

In [ ]:
type(data)

In [ ]:
len(data)

In [ ]:
data[0]

In [ ]:
data[1]

In [ ]:
print(data[3].page_content)

In [ ]:
# imprimir estructuras complejas en forma organizada
from pprint import pprint

n = 3 # visualizar los tres primeros documentos (filas)
for doc in data[:n]:
    pprint({
        "page_content": doc.page_content,
        "metadata": doc.metadata
    })


## Cargador de datos HTML

In [ ]:
# pip install beautifulsoup4
# pip install lxml  # Parser de respaldo

from langchain.document_loaders import BSHTMLLoader 

In [ ]:
loader = BSHTMLLoader('Datos/IngenieriaSistemasUP.html')


In [ ]:
data = loader.load()

In [ ]:
len(data)

In [ ]:
print(data[0].page_content)

In [ ]:
#print(data[0].page_content)
print(data[0].metadata)

## Cargar contenidos de una URL en linea

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

#loader = WebBaseLoader("https://www.unipamplona.edu.co")

# opcionar para simular el browser de conexion

loader = WebBaseLoader(
    web_paths=["https://www.unipamplona.edu.co"],
    header_template={"User-Agent": "Mozilla/5.0 (compatible; MyBot/1.0)"}
)

data = loader.load()


In [ ]:
print(data[0].metadata)

In [ ]:
#print(data[0].page_content)

## Carga de datos en PDF

In [ ]:
# pip install pypdf en una terminal

from langchain.document_loaders import PyPDFLoader 

In [ ]:
loader = PyPDFLoader('Datos/TendenciasTecnologicas2025.pdf')

In [ ]:
# Una paguna un documento
pages = loader.load()

# opcional: Divide de acuerdo al parámetro CharacterTextSplitter
#from langchain.text_splitter import CharacterTextSplitter
#from langchain_community.document_loaders import TextLoader

#splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50, separator=" ")
#pages = loader.load_and_split(text_splitter=splitter)


In [ ]:
type(pages)

In [ ]:
len(pages)

In [ ]:
pages[0].metadata

In [ ]:
print(pages[0].page_content)

## Caso de uso: Resumir PDF's

### Importar librerías e instancia de modelos de chat

In [ ]:
from langchain.prompts import (PromptTemplate, 
                                SystemMessagePromptTemplate, 
                                ChatPromptTemplate, 
                                HumanMessagePromptTemplate)

### Cargamos el modelo, en este caso de Ollama, puede modificarse según su preferencia

In [ ]:
# importamos las clases para manejar conversaciones con modelos de Ollama
from langchain_ollama.chat_models import ChatOllama

### Instaciamos un chat con uno de los modelos: llama3.2:3b, mistral:latest, gema3:4b, o los que se hayan instalado en Ollama

#chat = ChatOllama(model="llama3.2:3b")
#chat = ChatOllama(model="mistral:latest")
chat = ChatOllama(model="gemma3:4b")


In [ ]:
contenido_pdf = pages[0].page_content
contenido_pdf

In [ ]:
human_template = '"Necesito que hagas un resumen del siguiente texto en 500 caracteres: \n{contenido}"'
human_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])
#chat_prompt.format_prompt(contenido = contenido_pdf)

In [ ]:
solicitud = chat_prompt.format_prompt(contenido = contenido_pdf).to_messages()

In [ ]:
#solicitud

In [ ]:
result = chat.invoke(solicitud)

In [ ]:
print(result.content)

### Resumen del documento completo

- Si queremos resumir el documento completo, Creamos un string concatendando el contenido de todas las páginas:

In [ ]:

documento_completo = ""
for page in pages:
    documento_completo += page.page_content


In [ ]:
#print(documento_completo)

In [ ]:
human_template = '"Realizar un resumen del texto dado a continuación. Importante!!: El texto generado debe tener más de  300 palabras \n{contenido}"'
human_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [ ]:
nueva_solicitud = chat_prompt.format_prompt(contenido = documento_completo).to_messages()

In [ ]:
resultado = chat.invoke(nueva_solicitud)


In [ ]:
print(resultado.content)

In [ ]:
# Contemos las palabras

len(resultado.content.split())